# Nonlinear Objective function

The objective function for the one step ahead method is:
$$
J_{os} = c_s s(k) + c_b b(k+1)
$$

where $$b(k+1) = \frac{L(k+1)}{L(k+1) + Z(k+1)}$$ is the balking rate,$s(k)$ is the number of servers and $c_s$ and $c_b$ are coefficients that need to be chosen.

And for the omniscient model is:
$$
J_{om} = c_s\sum_{k}{s(k)} + c_b\sum_{k}{b(k+1)}
$$

## Size of the demand set and horizon

In [1]:
N = 10; # number of example scenarios 
horiz = 20;
createDemands = false;
# createDemands = true;

## Define structures: problem parameters & demand and abandonment sequences

In [2]:
using DelimitedFiles

module demand_stats
    using DelimitedFiles
    using Dates
    #Matrix demand generator: d_mat = demand_generator_mat(N_dem,demand_length,dM,demand_type,std_dev)
    include("demand_generator_mat.jl")
    N_dem  = Main.N;
    demand_length = Main.horiz
    dM = 14 #maximum demand
    aM = 2  #maximum abandonment
    demand_type = "uniform" #Can be normal ou Poisson
    std_dev_d = 1
    std_dev_a = 0.5
    limit_count = 10 # max number of trials to create a new replacement demand in case of infeasibility

    # Create demands or load them from files
    d_mat = zeros(demand_length,N_dem)
    a_mat = zeros(demand_length,N_dem)
    if Main.createDemands #create demands
        d_mat = demand_generator_mat(N_dem,demand_length,dM,demand_type,std_dev_d) #demand_generator_mat(3,20,14,"uniform",1)
        a_mat = demand_generator_mat(N_dem,demand_length,aM,demand_type,std_dev_a)
        # Output file names
        date = Dates.format(Dates.now(), "e_dd_u_yyyy_HH_MM_SS")
        d_mat_fn = "d_mat_" * string(date) * ".txt";
        a_mat_fn = "a_mat_" * string(date) * ".txt";

        writedlm(d_mat_fn, d_mat)
        writedlm(a_mat_fn, a_mat)

        # Print output file names
        println("Data saved to files:\n", d_mat_fn, "\n", a_mat_fn, "\n")

    else #load demands from file
        #d_fn = "d_mat_Thu_29_Aug_2024_12_35_06.txt";
        #a_fn = "a_mat_Thu_29_Aug_2024_12_35_06.txt";
    
        d_fn = "d_mat_Thu_22_May_2025_19_39_50.txt";
        a_fn = "a_mat_Thu_22_May_2025_19_39_50.txt";

        d_mat = DelimitedFiles.readdlm(d_fn);
        a_mat = DelimitedFiles.readdlm(a_fn);

        println("\nLoading demands from files:\n", d_fn, "\n", a_fn, "\n")
    end
end

# Extracts matrices from module demand_stats
d_mat = demand_stats.d_mat;
a_mat = demand_stats.a_mat;

println("d_mat = ", d_mat)
println("a_mat = ", a_mat)


Loading demands from files:
d_mat_Thu_22_May_2025_19_39_50.txt
a_mat_Thu_22_May_2025_19_39_50.txt

d_mat = [7.0 7.0 13.0 13.0 13.0 14.0 9.0 3.0 6.0 2.0; 3.0 11.0 13.0 7.0 3.0 3.0 3.0 13.0 3.0 7.0; 3.0 11.0 13.0 7.0 11.0 6.0 3.0 12.0 10.0 8.0; 7.0 11.0 5.0 12.0 13.0 13.0 2.0 5.0 2.0 3.0; 12.0 13.0 7.0 4.0 4.0 11.0 10.0 2.0 6.0 1.0; 9.0 3.0 3.0 7.0 2.0 3.0 12.0 1.0 14.0 12.0; 6.0 4.0 13.0 9.0 9.0 11.0 8.0 7.0 11.0 10.0; 3.0 12.0 3.0 12.0 3.0 2.0 3.0 7.0 5.0 4.0; 0.0 8.0 10.0 11.0 6.0 4.0 1.0 8.0 14.0 9.0; 9.0 1.0 3.0 10.0 14.0 2.0 3.0 1.0 7.0 2.0; 11.0 5.0 12.0 5.0 1.0 5.0 4.0 14.0 4.0 0.0; 12.0 12.0 5.0 10.0 6.0 9.0 4.0 8.0 6.0 10.0; 12.0 13.0 4.0 5.0 2.0 4.0 12.0 8.0 0.0 2.0; 12.0 12.0 9.0 13.0 2.0 10.0 12.0 12.0 6.0 13.0; 2.0 10.0 10.0 13.0 9.0 12.0 9.0 7.0 13.0 1.0; 0.0 10.0 12.0 4.0 6.0 8.0 9.0 1.0 3.0 1.0; 6.0 12.0 4.0 7.0 8.0 9.0 7.0 2.0 7.0 1.0; 13.0 10.0 11.0 7.0 11.0 7.0 8.0 5.0 2.0 11.0; 6.0 10.0 4.0 7.0 9.0 9.0 4.0 1.0 2.0 6.0; 13.0 9.0 1.0 1.0 2.0 13.0 14.0 0.0 8.0 9.0]
a_m

In [3]:
include("CCParameters.jl")

module CC_stats_nl_fobj
    include("CC_nl_fobj_os.jl")
    include("CC_nl_fobj_om.jl")
    include("CC_compare_stats_nl_model.jl")  #Call with method = "os"
    #include("CC_nl_fobj_plot2.jl")
    #include("CC_Plots.jl")
    #include("CC_stats_os_tables.jl")
    #include("CC_stats_vs_cb.jl")
    #include("gen_cb_stats_fixed_demand_os.jl")
end

Main.CC_stats_nl_fobj

## One Step Ahead

In [4]:
@time (X_os, Y_os, s_os, b_os) = CC_stats_nl_fobj.CC_compare_stats_nl_model(CCParameters.ic,CCParameters.bds,
    CCParameters.exd, CCParameters.c_ser, CCParameters.c_blr, demand_stats,"os")


Demand #1

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Demand #2
Demand #3
Demand #4
Demand #5
Demand #6
Demand #7
Demand #8
Demand #9
Demand #10
 56.891013 seconds (57.75 M allocations: 2.752 GiB, 3.00% gc time, 82.80% compilation time: 22% of which was recompilation)


([4.0 4.0 … 4.0 4.0; 4.999999945584401 0.0 … 0.0 0.0; … ; 5.0 5.0 … 5.0 5.0; 5.0 5.0 … 5.0 5.0], [2.0 2.0 … 2.0 2.0; 8.000000028877546 7.0 … 6.0 2.0; … ; 10.0 10.0 … 10.0 10.0; 10.0 10.0 … 10.0 10.0], [-6.98954893478676e-9 5.0 … 5.0 4.0; 5.0 5.0 … 5.0 2.0; … ; 4.0 0.0 … 2.0 5.0; 5.0 0.0 … 5.0 5.0], [0.0 0.0 … 0.0 0.0; -9.981822121663484e-9 0.14285714285714285 … 0.14285714285714285 0.2857142857142857; … ; 0.432 0.5808383233532934 … 0.35398230088495575 0.4; 0.4492753623188406 0.6022727272727273 … 0.35537190082644626 0.40384615384615385])

In [5]:
X_ost = "X_os.txt";
writedlm(X_ost, X_os)

In [6]:
# Results for diferent balking rate costs
cb = [140 150 160]
# CC_stats_nl_fobj.CC_stats_vs_cb(cb, CCParameters.ic, CCParameters.bds, CCParameters.exd, CCParameters.c_ser, demand_stats)

1×3 Matrix{Int64}:
 140  150  160

## Omniscient

In [ ]:
@time (X_om, Y_om, s_om, b_om) = CC_stats_nl_fobj.CC_compare_stats_nl_model(CCParameters.ic,CCParameters.bds,
    CCParameters.exd, CCParameters.c_ser, CCParameters.c_blr, demand_stats,"om")

Demand #1
Demand #2
Demand #3
Demand #

In [ ]:
X_omt = "X_om.txt";
writedlm(X_omt, X_om)

In [ ]:
# CC_stats_nl_fobj.CC_stats_vs_cb(cb, CCParameters.ic, CCParameters.bds, exd, CCParameters.c_ser, CCParameters.c_blr, demand_stats)